General task : 

   - The goal of this task is to create personalized recommendations for football matches
   (events) for users over a one-week period. Personalized recommendations are crucial in
   numerous applications to ensure users are satisfied with the content provided. In this case,
   you will generate personalized recommendations of football matches for a one-week period
   based on the teams each user follows.

Data retrival

In [18]:
from clickhouse_driver import Client
import pandas as pd
import ast

In [19]:
bqClient = Client(
user='mharalovic',
password='Fs75EePJ3m54EyysB75U',
host='clickhouse.sofascore.ai',
port='9000',
)

Retriving all football teams that played a match between '20230101' and '20230630' from the table sports.event.

In [20]:
query = """    SELECT t.id
               FROM sports.event AS e
               LEFT JOIN sports.sport AS s ON e.sport_id = toInt8(s.id)
               LEFT JOIN sports.team AS t ON e.hometeam_id = t.id
               WHERE s.name = 'Football'
               AND toYYYYMMDD(e.startdate) BETWEEN '20230101' AND '20230630'
               UNION DISTINCT
               SELECT t1.id
               FROM sports.event AS e
               LEFT JOIN sports.sport AS s ON e.sport_id = toInt8(s.id)
               LEFT JOIN sports.team AS t1 ON e.awayteam_id = t1.id
               WHERE s.name = 'Football'
               AND toYYYYMMDD(e.startdate) BETWEEN '20230101' AND '20230630';
       """

In [21]:
team_ids = bqClient.execute(query)

In [22]:
teams_df = pd.DataFrame(team_ids, columns=['team_id'])
teams_df.to_csv('teams.csv', index=False)

Retrieving all users data 

In [23]:
query2 = """ SELECT user_account_id, teams, mcc
            FROM bq.mobileuser
            WHERE user_account_id IS NOT NULL
               AND length(teams) > 0
               AND toYYYYMMDD(created_at) <= '20230630'
               AND toYYYYMMDD(updated_at) <= '20230630'
               AND mcc IN (216, 218, 219, 220, 221, 222, 226, 232, 262, 276, 284, 293, 294,297)
            ORDER BY mcc DESC
        """

In [24]:
user_data =  bqClient.execute(query2)

In [25]:
user_data_df = pd.DataFrame(user_data, columns=['user_account_id', 'teams', 'mcc'])
user_data_df['teams'] = user_data_df['teams'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


In [26]:
user_data_df.to_csv('user_data.csv',index=False)

In [27]:
user_data_df.head()

,user_account_id,teams,mcc
0,5bc624c559a182e430b5c8b4,"[5152, 14882, 6637]",297
1,637e42cdd8b5f7f22307c1a8,"[6355, 4715]",297
2,5e31ec0f6dbf6b00f30debe3,"[5152, 6637]",297
3,6256b62333e7907d55c25836,"[8114, 224677, 193686, 187295, 194497, 6635, 7...",297
4,62362f28b2e2666e19221d7b,"[4481, 4739, 1925, 2693, 6023, 2952, 1289, 220...",297


Data preprocessing tasks 
- filter the teams that each user follows by retaining only those teams that are playing relevant events
- if a user does not follow any of those teams, discard the user

1st task -> getting relevant teams

In [28]:
user_data_df['teams'] = user_data_df['teams'].apply(lambda team_list: [team for team in team_list if team in teams_df['team_id'].values])

In [29]:
user_data_df = user_data_df[user_data_df['teams'].apply(len) > 0]

- Grouping user by MCC and getting the teams that each user follow
- Creating dict afterwards -> key is mcc, values are 'teams' and 'user_account_id', which hold all the inf about the teams and user account connected to specific group (mcc) -> for O(1) lookups

In [41]:
grouped_by_mcc = user_data_df.groupby('mcc').agg({
    'user_account_id': lambda x: list(x),
    'teams': lambda x: list(set(sum(x, [])))
}).reset_index().set_index('mcc')

In [59]:
grouped_by_mcc = grouped_by_mcc.apply(lambda row: row.update({'user_account_id': sorted(row['user_account_id']), 'teams': sorted(row['teams'])}) or row, axis=1)

In [60]:
grouped_by_mcc_dict = grouped_by_mcc.to_dict(orient='index')

Saving dict and dataframe locally

In [67]:
import pickle 

with open('grouped_by_mcc_dict.pkl', 'wb') as f:
    pickle.dump(grouped_by_mcc_dict, f)

In [66]:
user_data_df.to_csv('user_data.csv',index=False)

Retrieving events data

In [72]:
query3 = """
        SELECT 
            toStartOfWeek(startdate, 1) AS week_start,
            groupArray(id) AS event_ids,
            groupArray((hometeam_id, awayteam_id)) AS team_ids
         FROM
            sports.event
         LEFT JOIN
            sports.sport s ON event.sport_id = toInt8(s.id)
         WHERE
            toYYYYMMDD(startdate) >= '20230601' AND toYYYYMMDD(startdate) <= '20230630' AND s.name = 'Football'
         GROUP BY 
            week_start
         ORDER BY 
            week_start
        """

In [76]:
events_data = bqClient.execute(query3)

In [77]:
events_data = pd.DataFrame(events_data, columns=['week_start', 'event_ids', 'team_ids'])

[(datetime.date(2023, 5, 29),
  [11016199,
   11016200,
   11016201,
   11016202,
   11016203,
   11016205,
   11016329,
   11016330,
   11016331,
   11016332,
   11016333,
   11016698,
   11016701,
   11016705,
   11016711,
   11016713,
   11016717,
   11016892,
   11016897,
   11016904,
   11016909,
   11016914,
   11016920,
   11016976,
   11016994,
   11017001,
   11017009,
   11017210,
   11017214,
   11017218,
   11017222,
   11017226,
   11017229,
   11017324,
   11017327,
   11017330,
   11017332,
   11017335,
   11017338,
   11017406,
   11017408,
   11017410,
   11017413,
   11017415,
   11018731,
   11018732,
   11018733,
   11018734,
   11020336,
   11020337,
   11020338,
   11020341,
   11020344,
   11020349,
   11020350,
   11020351,
   11020653,
   11020657,
   11020658,
   11020659,
   11020660,
   11020661,
   11020662,
   11020663,
   11020664,
   11020665,
   11020827,
   11020829,
   11020830,
   11020831,
   11020832,
   11020833,
   11020834,
   11020835,
   11020